In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../app')

In [2]:
from data_retrieval import vertical_download, reconstruction, download_indicators, dfs_list_from_dir

import pandas as pd
import eikon as ek

import app_functions as af
import custom_calculations as calcs

### Downloading

In [3]:
instruments_list = ek.get_data('0#.STOXX','TR.RIC')[0].RIC.tolist()

In [29]:
#las funciones de eikon a descargar
fields_list = ['TR.TRESGScore',
               #'TR.intexpensenetnonoper'
               ]
#parametros de la funcion
parameters = {'SDate':'2022-01-01','EDate':'2023-01-01','Frq':'FY','Period':'FY0'}
#directorio en el que guardar los datos
dir_fp = r'C:\Users\hugo.perezdealbeniz\Documents\GitHub\ranking_DIP_european_equities\eikon_data_retrieval\data'

In [30]:
len(instruments_list)

600

In [ ]:
data_dict = download_indicators(fields_list,instruments_list,parameters,dir_fp)

In [18]:
df,err = ek.get_data('0#.STOXX',['TR.TRESGScore','TR.TRESGScore.date'],parameters)
df

,Instrument,ESG Score,Date
0,BOUY.PA,70.925529,2010-12-31T00:00:00Z
1,BOUY.PA,59.769613,2011-12-31T00:00:00Z
2,BOUY.PA,57.794933,2012-12-31T00:00:00Z
3,BOUY.PA,57.113348,2013-12-31T00:00:00Z
4,BOUY.PA,59.012239,2014-12-31T00:00:00Z
...,...,...,...
6234,ENI.MI,83.525107,2017-12-31T00:00:00Z
6235,ENI.MI,84.279577,2018-12-31T00:00:00Z
6236,ENI.MI,82.146774,2019-12-31T00:00:00Z
6237,ENI.MI,83.778623,2020-12-31T00:00:00Z


In [19]:
clean_df = df.replace(['','NaN',pd.NA]).dropna().drop_duplicates()

In [10]:
data_dict.keys()

dict_keys(['TRESGScore'])

### Calculations and ranking

In [ ]:
n_quantiles = 10

In [11]:
esg_df = data_dict['TRESGScore']

In [22]:
pivoted_df = clean_df.pivot(columns='Instrument',index='Date',values='ESG Score')

In [ ]:
pivoted_df.fillna(method='fill',limit=12)

,1COV.DE,ABBN.S,ABNd.AS,AD.AS,ADDTb.ST,ADML.L,ADSGn.DE,AFXG.DE,AHT.L,AIRP.PA,...,VMUK.L,VOD.L,VOES.VI,VOLVb.ST,VOWG_p.DE,VTWRn.DE,VWS.CO,WISEa.L,WOSG.L,WRT1V.HE
2022-01-31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-02-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-03-31,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-04-30,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,54.785907,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-05-31,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,54.785907,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-06-30,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,54.785907,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-07-31,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,54.785907,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-08-31,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,<NA>,54.785907,<NA>,...,<NA>,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-09-30,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,71.894283,54.785907,<NA>,...,81.48921,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>
2022-10-31,<NA>,<NA>,<NA>,<NA>,42.253936,<NA>,<NA>,71.894283,54.785907,<NA>,...,81.48921,88.037446,66.593369,<NA>,<NA>,31.996713,<NA>,33.649957,<NA>,<NA>


In [28]:
pivoted_df.dropna(axis=1,how='all')

Instrument,1COV.DE,A2.MI,AAK.ST,AAL.L,AALB.AS,ABBN.S,ABDN.L,ABF.L,ABI.BR,ABNd.AS,...,WISEa.L,WLN.PA,WLSNc.AS,WOSG.L,WPP.L,WRT1V.HE,WTB.L,YAR.OL,ZALG.DE,ZURN.S
Date,,,,,,,,,,,,,,,,,,,,,
2009-02-26T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,61.944362,<NA>,<NA>,<NA>
2009-03-21T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-03-28T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-03-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-07-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-10-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-11-27T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [27]:
pivoted_df

Instrument,1COV.DE,A2.MI,AAK.ST,AAL.L,AALB.AS,ABBN.S,ABDN.L,ABF.L,ABI.BR,ABNd.AS,...,WISEa.L,WLN.PA,WLSNc.AS,WOSG.L,WPP.L,WRT1V.HE,WTB.L,YAR.OL,ZALG.DE,ZURN.S
Date,,,,,,,,,,,,,,,,,,,,,
2009-02-26T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,61.944362,<NA>,<NA>,<NA>
2009-03-21T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-03-28T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-03-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2009-07-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-10-31T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2022-11-27T00:00:00Z,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
